## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意（`ORDER BY` は付けない）
* `NOT IN` は使用しない（本問はそもそも不要）
* 判定は `product_id` × `year`、表示は仕様どおり（`product_id, first_year, quantity, price`）

---

## 1) 問題

### PROBLEM_STATEMENT

Table: `Sales`

```text
+-------------+-------+
| Column Name | Type  |
+-------------+-------+
| sale_id     | int   |
| product_id  | int   |
| year        | int   |
| quantity    | int   |
| price       | int   |
+-------------+-------+
(sale_id, year) は複合主キー
```

* 各行は「ある `product_id` の、ある `year` における 1 件の販売」を表す
* 同じ `product_id`・同じ `year` で複数行あり得る

### 要求仕様

* 各 `product_id` ごとに、**その商品が最初に販売された年（最小の `year`）** を求める
* その「最初の年」に属する **全ての販売行** を抽出する
* 出力列:

  * `product_id`
  * `first_year` … その商品の最初の販売年
  * `quantity`
  * `price`
* 並び順: 任意

---

## 2) 最適解（単一クエリ / CTE + ウィンドウ）

PostgreSQL なので、素直に **`DENSE_RANK()` ウィンドウ関数**を使い、
「各 `product_id` 内で最も早い `year` の行だけを残す」形にします。

```sql
WITH win AS (
  SELECT
    product_id,
    year,
    quantity,
    price,
    DENSE_RANK() OVER (
      PARTITION BY product_id
      ORDER BY year
    ) AS rnk
  FROM Sales
)
SELECT
  product_id,
  year AS first_year,
  quantity,
  price
FROM win
WHERE rnk = 1;

Runtime 942 ms
Beats 43.80%

```

### テンプレートへの対応関係

```sql
WITH win AS (
  SELECT
    product_id,
    year,
    quantity,
    price,
    DENSE_RANK() OVER (
      PARTITION BY product_id         -- {{PARTITION_KEYS}}
      ORDER BY year                   -- {{ORDER_KEYS}}
    ) AS rnk                          -- {{RNK_OR_VAL}}
  FROM Sales                          -- {{PRE_OR_INPUT}}
)
SELECT
  product_id,
  year AS first_year,                 -- {{FINAL_PROJECTION}}
  quantity,
  price
FROM win
WHERE rnk = 1;                        -- {{FILTER_ON_WINDOW_OR_CONDITION}}

Runtime 878 ms
Beats 67.56%

```

* `pre` CTE は不要なため省略し、いきなり `win` CTE でウィンドウ処理を実施
* 「最初の年」の全行を残したいので `DENSE_RANK()` を使用

  * 同一 `product_id`・同一 `year` に複数行あっても、すべて `rnk = 1` になり仕様どおり

### 代替: LATERAL は本問では不要

提示テンプレートの「LATERAL + LIMIT k」パターンは

* 「各グループから上位 k 件だけ取りたい」
* 「行数が多いので、各グループについて少量だけ抜きたい」

といった場面で有効です。

本問は単に「最初の年の全行」を取りたいだけなので、
`DENSE_RANK()` or 集約 + 結合 で十分であり、`LATERAL` の出番はありません。

---

## 3) 要点解説

### ロジックの流れ

1. `Sales` をそのまま入力とする
2. `product_id` ごとに `year` 昇順で並べ、`DENSE_RANK()` を付与

   * 最小の `year` に `rnk = 1`
   * それ以外の年は `rnk >= 2`
3. `rnk = 1` の行だけを残す

   * その結果、「各 `product_id` が最初に売れた年」に属する全行が残る
4. `year` を `first_year` として出力し、他列はそのまま

### `DENSE_RANK` を採用している理由

* `ROW_NUMBER()` を使うと、「最初の年の中の 1 行だけ」しか残せない
* 本問は「最初の年の全ての販売行」を残したいので、

  * 同じ `year` の行をすべて `1` 位扱いにする `DENSE_RANK()` が適切

### `NOT IN` を避けている点

* 本問では「最小年以外の年を除外する」ために `NOT IN` / `IN` を使う必要がない
* ウィンドウ関数と `WHERE rnk = 1` で表現しているため、
  NULL 罠 (`NOT IN (subquery)` + NULL) とは無縁の書き方になっている

---

## 4) 計算量（概算）

`N` = `Sales` の行数、`G` = `product_id` のグループ数とします。

* ウィンドウ部（`DENSE_RANK() OVER (PARTITION BY product_id ORDER BY year)`）

  * 各グループごとに `year` でソートが入るため、
  * 時間計算量は概ね **O(Σ n_g log n_g)** ≒ **O(N log (N/G))**
* CTE からの最終 SELECT + `WHERE rnk = 1`

  * フィルタのみのため、**O(N)** 近似

総じて **O(N log N)** クラスで、典型的なウィンドウ処理のコスト感です。

インデックス面では、実務なら

```sql
CREATE INDEX ON Sales (product_id, year);
```

のような複合インデックスがあると、ソート負荷がかなり軽減されます
（`PARTITION BY product_id ORDER BY year` にぴったり合うため）。

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 Sales テーブル]
  B[ウィンドウ処理 product_id ごとに year で DENSE_RANK]
  C[rnk が 1 の行だけを抽出]
  D[列名を整形 product_id first_year quantity price]
  E[出力]

  A --> B
  B --> C
  C --> D
  D --> E
```

PostgreSQL 16.6+ 前提であれば、このウィンドウ関数版が最も素直で可読性も高く、
パフォーマンスとメンテナンス性のバランスが良い解法になります。

結論から言うと、

* 今のクエリは **ロジックとしては最適クラス**
* Runtime の 942ms → 878ms（67.56%）も、LeetCode 的には十分良好
* それでも「もう少し詰める」としたら、**ウィンドウ関数の形を変える**か、**集約＋結合パターン**を試す、という「微調整レベル」

くらいの温度感です。

そのうえで、現行案をベースに「変え得るポイント」を整理します。

---

## 1. 現行クエリの評価

```sql
WITH win AS (
  SELECT
    product_id,
    year,
    quantity,
    price,
    DENSE_RANK() OVER (
      PARTITION BY product_id
      ORDER BY year
    ) AS rnk
  FROM Sales
)
SELECT
  product_id,
  year AS first_year,
  quantity,
  price
FROM win
WHERE rnk = 1;
```

* アルゴリズム的には:

  * 各 `product_id` 内で `year` 昇順に順位付け → 最小年の行だけ残す
  * 「最初の年に属する全行」を残すので `DENSE_RANK()` 採用は正解
* 計算量は典型的なウィンドウ処理で **O(N log N)** クラス
* Runtime が 878ms / Beats 67.56% なら、
  「十分早い・無理にいじるほどではない」というラインです

→ ここからは完全に「チューニング遊び」の領域です。

---

## 2. 同じウィンドウ路線での微調整案

### 2-1. `MIN(year) OVER` を使う書き換え

`DENSE_RANK()` の代わりに、**パーティション内の最小年を直接持たせる**パターンです。

```sql
WITH win AS (
  SELECT
    product_id,
    year,
    quantity,
    price,
    MIN(year) OVER (
      PARTITION BY product_id
    ) AS first_year
  FROM Sales
)
SELECT
  product_id,
  first_year,
  quantity,
  price
FROM win
WHERE year = first_year;

Runtime 920 ms
Beats 51.45%

```

ポイント:

* `MIN(year) OVER (PARTITION BY product_id)` なので、

  * 各行に「その product_id の最小年」を付与
  * それと `year` が等しい行だけ残す
* `ORDER BY` なしのウィンドウなので、理論上は

  * 「ランク計算」よりは軽いことが多い
  * 実装上は結局ソートが入ることもありますが、オプティマイザに若干有利な場合があります

LeetCode 環境では差が出ないか、数％レベルの違いにとどまる可能性が高いですが、
**読みやすさ＋若干のコスト低減**という意味で試す価値はあります。

### 2-2. CTE をやめてサブクエリにする

PostgreSQL 12 以降は CTE は「最適化フェンス」ではなくなりましたが、
それでも素直にインラインの方がプランが分かりやすい場合があります。

```sql
SELECT
  product_id,
  first_year,
  quantity,
  price
FROM (
  SELECT
    product_id,
    year AS first_year,
    quantity,
    price,
    DENSE_RANK() OVER (
      PARTITION BY product_id
      ORDER BY year
    ) AS rnk
  FROM Sales
) t
WHERE rnk = 1;

Runtime 912 ms
Beats 54.81%

```

もしくは `MIN OVER` 版と組み合わせて:

```sql
SELECT
  product_id,
  first_year,
  quantity,
  price
FROM (
  SELECT
    product_id,
    year,
    quantity,
    price,
    MIN(year) OVER (PARTITION BY product_id) AS first_year
  FROM Sales
) t
WHERE year = first_year;

Runtime 951 ms
Beats 40.44%

```

LeetCode のような簡易環境だと **ほとんど誤差**ですが、
実務では「CTE をパターン的に書きすぎない」こともプラン安定の一助になります。

---

## 3. 代替パターン：集約 + 結合

MySQL 版でも触れたパターンですが、PostgreSQL でも王道です。

```sql
WITH first_years AS (
  SELECT
    product_id,
    MIN(year) AS first_year
  FROM Sales
  GROUP BY product_id
)
SELECT
  s.product_id,
  f.first_year,
  s.quantity,
  s.price
FROM Sales AS s
JOIN first_years AS f
  ON  s.product_id = f.product_id
  AND s.year       = f.first_year;

Runtime 965 ms
Beats 37.35%

```

特徴:

* `GROUP BY` で `product_id` ごとの最小年を先に確定させる
* その結果（1行/商品）を元テーブルに結合し、該当年の全行を取得
* 計算量としてはやはり **O(N log N)** クラスですが、

  * `GROUP BY` の実装（Hash Aggregate / Sort Aggregate）や
  * 結合戦略（Hash Join / Merge Join）
    によっては、ウィンドウ版より速くなることもあります

LeetCode の結果としても、MySQL 側ではこのパターンの方が若干速かったので、
PostgreSQL 版でも同じく「微差で有利」になる可能性はあります。

---

## 4. さらに簡潔な相関サブクエリ版（好みレベル）

もう一つよく使われる書き方として、**相関サブクエリで MIN(year)** を直接当てる形もあります。

```sql
SELECT
  s.product_id,
  s.year AS first_year,
  s.quantity,
  s.price
FROM Sales AS s
WHERE s.year = (
  SELECT MIN(year)
  FROM Sales AS s2
  WHERE s2.product_id = s.product_id
);

Runtime 2942 ms
Beats 5.04%
```

* ロジックが非常に読みやすい
* PostgreSQL は相関サブクエリをかなりうまくデコリレーションしてくれるので、

  * 実行計画としては「GROUP BY＋結合」に近い形になることが多い
* インデックスが `(product_id, year)` に張られていれば、
  実務ではかなり優秀なパターンです

LeetCode 環境でも、これを試してみて Runtime がどう変わるかを見る価値はあります。

---

## 5. まとめ

* **今のウィンドウ版（DENSE_RANK）は十分良い**です

  * 878ms / Beats 67.56% という数値からも、無理にいじる必要はないレベル
* それでも「もう一歩攻める」なら：

  1. `DENSE_RANK()` → `MIN(year) OVER (PARTITION BY product_id)` に書き換え
  2. CTE をインラインサブクエリにしてみる
  3. `GROUP BY + JOIN` 版
  4. 相関サブクエリ版

  をそれぞれ試して、Runtime のブレを見比べる、くらいが現実的です。

ここまで来るとほぼ「チューニング道楽」の世界なので、
この問題に関しては「今のクエリを PG16 版のテンプレ解」として採用しておいて問題ないと思います。
